# 🔬 Light Calculator Dashboard

**Calculate electrons per pixel in camera sensors**

This interactive calculator follows light from scene illumination through the lens to the final electron count in sensor pixels. Adjust the parameters below to explore how different camera settings and conditions affect light collection.

---

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, VBox, HBox, Layout
import warnings
warnings.filterwarnings('ignore')

# Import our light calculator functions
from light_calculator import (
    calculate_full_chain, 
    format_results_for_notebook,
    get_calculation_data_for_plotting
)

# Configure matplotlib for better dashboard appearance
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14

## 🎛️ Camera Parameters

Adjust the sliders below to see how different settings affect light collection:

In [ ]:
# Create styled widgets for dashboard
widget_layout = Layout(width='500px')
slider_style = {'description_width': '160px'}

# Scene parameters
scene_illuminance = widgets.FloatLogSlider(
    value=1000.0,
    base=10,
    min=0,  # 10^0 = 1
    max=5,  # 10^5 = 100,000
    step=0.1,
    description='💡 Scene Illuminance (lux):',
    style=slider_style,
    layout=widget_layout,
    readout_format='.0f'
)

scene_reflectance = widgets.FloatSlider(
    value=18.0,
    min=1.0,
    max=90.0,
    step=1.0,
    description='🎯 Scene Reflectance (%):',
    style=slider_style,
    layout=widget_layout,
    readout_format='.0f'
)

# Lens parameters
f_number = widgets.FloatSlider(
    value=2.8,
    min=1.0,
    max=22.0,
    step=0.1,
    description='📷 Lens f-number:',
    style=slider_style,
    layout=widget_layout,
    readout_format='.1f'
)

lens_transmittance = widgets.FloatSlider(
    value=85.0,
    min=60.0,
    max=95.0,
    step=1.0,
    description='🔍 Lens Transmittance (%):',
    style=slider_style,
    layout=widget_layout,
    readout_format='.0f'
)

# Sensor parameters
pixel_size = widgets.FloatSlider(
    value=5.0,
    min=1.0,
    max=15.0,
    step=0.1,
    description='📐 Pixel Size (μm):',
    style=slider_style,
    layout=widget_layout,
    readout_format='.1f'
)

exposure_time = widgets.FloatLogSlider(
    value=16.67,
    base=2,
    min=-1,  # 2^-1 = 0.5
    max=10,   # 2^10 = 1024
    step=0.05,
    description='⏱️ Exposure Time (ms):',
    style=slider_style,
    layout=widget_layout,
    readout_format='.1f'
)

wavelength = widgets.FloatSlider(
    value=550.0,
    min=400.0,
    max=700.0,
    step=10.0,
    description='🌈 Wavelength (nm):',
    style=slider_style,
    layout=widget_layout,
    readout_format='.0f'
)

quantum_efficiency = widgets.FloatSlider(
    value=60.0,
    min=20.0,
    max=95.0,
    step=1.0,
    description='⚡ Quantum Efficiency (%):',
    style=slider_style,
    layout=widget_layout,
    readout_format='.0f'
)

# Noise parameters
read_noise = widgets.FloatSlider(
    value=3.0,
    min=0.1,
    max=20.0,
    step=0.1,
    description='🔊 Read Noise (e⁻):',
    style=slider_style,
    layout=widget_layout,
    readout_format='.1f'
)

dark_current = widgets.FloatLogSlider(
    value=0.1,
    base=10,
    min=-2,  # 10^-3 = 0.001
    max=2,   # 10^1 = 10
    step=0.1,
    description='🌡️ Dark Current (e⁻/px/s):',
    style=slider_style,
    layout=widget_layout,
    readout_format='.3f'
)

# Organize widgets in three columns
left_column = VBox([
    scene_illuminance,
    scene_reflectance,
    f_number,
    lens_transmittance
], layout=Layout(width='500px'))

middle_column = VBox([
    pixel_size,
    exposure_time,
    wavelength,
    quantum_efficiency
], layout=Layout(width='500px'))

right_column = VBox([
    read_noise,
    dark_current
], layout=Layout(width='500px'))

controls = HBox([left_column, middle_column, right_column])
display(controls)

---

## 📊 Results & Visualization

In [ ]:
# Create output widget for results
output = widgets.Output()

def update_calculation(*args):
    """Update calculation and display results"""
    with output:
        clear_output(wait=True)
        
        # Convert percentages to ratios
        scene_reflectance_ratio = scene_reflectance.value / 100.0
        lens_transmittance_ratio = lens_transmittance.value / 100.0
        quantum_efficiency_ratio = quantum_efficiency.value / 100.0
        
        # Perform calculation with noise analysis
        results = calculate_full_chain(
            scene_illuminance.value, scene_reflectance_ratio, lens_transmittance_ratio,
            f_number.value, pixel_size.value, exposure_time.value, 
            wavelength.value, quantum_efficiency_ratio,
            read_noise.value, dark_current.value
        )
        
        # Display formatted results (now includes noise analysis)
        display(format_results_for_notebook(results))
        
        # Create enhanced visualizations
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
        
        # 1. Light collection process (top left)
        res = results['results']
        step_names = ['Scene\nLuminance', 'Sensor\nIlluminance', 'Photon\nCount', 'Signal\nElectrons']
        step_values = [res['scene_luminance_nits'], res['sensor_illuminance_lux'], 
                      res['photon_count'], res['electron_count']]
        colors = ['#28a745', '#007bff', '#ffc107', '#6f42c1']
        
        bars = ax1.bar(step_names, step_values, color=colors, alpha=0.8)
        ax1.set_ylabel('Value (log scale)')
        ax1.set_title('🔄 Light Collection Process', fontweight='bold')
        ax1.set_yscale('log')
        ax1.grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar, value in zip(bars, step_values):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height * 1.5,
                    f'{value:,.0f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
        
        # 2. Noise breakdown (top right)
        noise = results['noise_analysis']
        noise_components = ['Shot\nNoise', 'Dark\nNoise', 'Read\nNoise', 'Total\nNoise']
        noise_values = [
            noise['noise_components']['shot_noise'],
            noise['noise_components']['dark_noise'], 
            noise['noise_components']['read_noise'],
            noise['noise_components']['total_noise']
        ]
        noise_colors = ['#28a745', '#dc3545', '#ffc107', '#6f42c1']
        
        bars2 = ax2.bar(noise_components, noise_values, color=noise_colors, alpha=0.8)
        ax2.set_ylabel('Noise (electrons)')
        ax2.set_title('🔊 Noise Analysis', fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        # Add noise value labels
        for bar, value in zip(bars2, noise_values):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + max(noise_values)*0.02,
                    f'{value:.1f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
        
        # 3. Signal vs Noise comparison (bottom left)
        snr_data = [res['electron_count'], noise['noise_components']['total_noise']]
        snr_labels = ['Signal\n(electrons)', 'Total Noise\n(electrons)']
        snr_colors = ['#198754', '#dc3545']
        
        bars3 = ax3.bar(snr_labels, snr_data, color=snr_colors, alpha=0.8)
        ax3.set_ylabel('Electrons')
        ax3.set_title(f'📊 SNR = {noise["snr"]["linear"]:.1f} ({noise["snr"]["db"]:.1f} dB)', fontweight='bold')
        ax3.set_yscale('log')
        ax3.grid(True, alpha=0.3)
        
        # Add SNR labels
        for bar, value in zip(bars3, snr_data):
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height * 1.2,
                    f'{value:,.0f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
        
        # 4. Noise regime indicator (bottom right)
        # Create a visual representation of noise regime
        regime_colors = {
            'shot_limited': '#28a745',
            'dark_limited': '#dc3545', 
            'read_limited': '#ffc107',
            'shot_dark_limited': '#17a2b8'
        }
        regime_color = regime_colors.get(noise['noise_regime'], '#6c757d')
        
        # Show relative contributions
        contributions = [
            noise['noise_components']['shot_noise']**2,
            noise['noise_components']['dark_noise']**2,
            noise['noise_components']['read_noise']**2
        ]
        contribution_labels = ['Shot²', 'Dark²', 'Read²']
        contribution_colors = ['#28a745', '#dc3545', '#ffc107']
        
        # Filter out zero contributions for cleaner pie chart
        non_zero_contrib = [(val, label, color) for val, label, color in 
                           zip(contributions, contribution_labels, contribution_colors) if val > 0.01]
        
        if non_zero_contrib:
            values, labels, colors = zip(*non_zero_contrib)
            wedges, texts, autotexts = ax4.pie(values, labels=labels, colors=colors, 
                                              autopct='%1.1f%%', startangle=90, 
                                              textprops={'fontweight': 'bold', 'fontsize': 9})
        
        regime_title = noise['noise_regime'].replace('_', ' ').title()
        ax4.set_title(f'⚙️ Noise Regime: {regime_title}', fontweight='bold', color=regime_color)
        
        plt.tight_layout()
        plt.show()

# Attach observers to all widgets (including new noise parameters)
all_widgets = [scene_illuminance, scene_reflectance, f_number, lens_transmittance,
               pixel_size, exposure_time, wavelength, quantum_efficiency, 
               read_noise, dark_current]

for widget in all_widgets:
    widget.observe(update_calculation, names='value')

# Initial calculation
update_calculation()

# Display output
display(output)

---

## 🎯 Quick Presets

Try these common photography scenarios:

In [ ]:
def apply_preset(illuminance, reflectance, f_num, transmittance, pixel, exposure, wave, qe, read_n, dark_c):
    """Apply preset values to all widgets including noise parameters"""
    scene_illuminance.value = illuminance
    scene_reflectance.value = reflectance
    f_number.value = f_num
    lens_transmittance.value = transmittance
    pixel_size.value = pixel
    exposure_time.value = exposure
    wavelength.value = wave
    quantum_efficiency.value = qe
    read_noise.value = read_n
    dark_current.value = dark_c

# Create preset buttons with enhanced noise scenarios
btn_daylight = widgets.Button(
    description='☀️ Bright Daylight',
    button_style='info',
    layout=Layout(width='160px', height='40px')
)

btn_portrait = widgets.Button(
    description='🏠 Indoor Portrait', 
    button_style='warning',
    layout=Layout(width='160px', height='40px')
)

btn_lowlight = widgets.Button(
    description='🌙 Low Light',
    button_style='danger',
    layout=Layout(width='160px', height='40px')
)

btn_astro = widgets.Button(
    description='🔭 Astro Camera',
    button_style='primary',
    layout=Layout(width='160px', height='40px')
)

btn_smartphone = widgets.Button(
    description='📱 Smartphone',
    button_style='dark',
    layout=Layout(width='160px', height='40px')
)

btn_reset = widgets.Button(
    description='🔄 Reset Default',
    button_style='success',
    layout=Layout(width='160px', height='40px')
)

# Enhanced button click handlers with noise parameters
def on_daylight_click(b):
    # Bright daylight: Good light, modern DSLR
    apply_preset(50000, 18, 8.0, 90, 4.0, 0.5, 550, 70, 2.5, 0.05)

def on_portrait_click(b):
    # Indoor portrait: Moderate light, full-frame camera
    apply_preset(2000, 25, 2.8, 85, 6.0, 16.67, 550, 65, 3.5, 0.1)

def on_lowlight_click(b):
    # Low light: Fast lens, large pixels, higher noise
    apply_preset(1, 10, 1.4, 80, 8.0, 1000, 550, 90, 5.0, 0.5)

def on_astro_click(b):
    # Cooled astronomy camera: Excellent noise performance
    apply_preset(0.01, 8, 2.0, 90, 9.0, 30000, 550, 90, 1.5, 0.001)

def on_smartphone_click(b):
    # Smartphone: Small pixels, higher noise floor
    apply_preset(1000, 18, 2.0, 80, 1.4, 33, 550, 50, 8.0, 2.0)

def on_reset_click(b):
    # Default: Typical DSLR settings
    apply_preset(1000, 18, 2.8, 85, 5.0, 16.67, 550, 60, 3.0, 0.1)

btn_daylight.on_click(on_daylight_click)
btn_portrait.on_click(on_portrait_click)
btn_lowlight.on_click(on_lowlight_click)
btn_astro.on_click(on_astro_click)
btn_smartphone.on_click(on_smartphone_click)
btn_reset.on_click(on_reset_click)

# Display buttons in two rows
top_row = HBox([btn_daylight, btn_portrait, btn_lowlight], 
               layout=Layout(justify_content='space-around'))
bottom_row = HBox([btn_astro, btn_smartphone, btn_reset], 
                  layout=Layout(justify_content='space-around'))

preset_box = VBox([top_row, bottom_row], layout=Layout(margin='10px 0'))
display(preset_box)

---

## 📚 Understanding Camera Noise & Image Quality

### 🔊 **Noise Sources Explained:**

**1. Shot Noise (Photon Noise)** 📸
- **Source**: Random arrival of photons (quantum physics)
- **Formula**: √(signal electrons)
- **Characteristics**: Unavoidable, increases with signal
- **Dominates**: In bright conditions with good cameras

**2. Read Noise** 🔊
- **Source**: Electronics in readout circuitry  
- **Formula**: Constant value (e.g., 3 electrons RMS)
- **Characteristics**: Independent of exposure time/light
- **Dominates**: In very low light conditions

**3. Dark Current Noise** 🌡️
- **Source**: Thermal generation of electrons
- **Formula**: √(dark_current × exposure_time)
- **Characteristics**: Increases with temperature and exposure
- **Dominates**: In long exposures or warm sensors

### 📊 **Signal-to-Noise Ratio (SNR)**
- **Formula**: Signal / Total_Noise
- **Good SNR**: >10 (20+ dB) for quality images
- **Excellent SNR**: >100 (40+ dB) for professional work
- **Critical**: Higher SNR = cleaner, more detailed images

### ⚙️ **Noise Regimes:**
- **🟢 Shot Limited**: Best case - quantum limited performance
- **🔴 Dark Limited**: Long exposures, warm sensors  
- **🟡 Read Limited**: Very low light, short exposures
- **🔵 Shot+Dark Limited**: Balanced noise sources

### 💡 **Optimization Tips:**

**For Better SNR:**
- ⬆️ **Increase light**: Higher illumination, faster lens, larger pixels
- ⬆️ **Longer exposure**: More signal (if not dark limited)
- ⬇️ **Cool sensor**: Reduces dark current (astronomy cameras)
- ⬇️ **Choose low-noise sensor**: Better read noise specs

**Typical Camera Performance:**
- **📱 Smartphone**: 5-15e⁻ read noise, 1-10e⁻/px/s dark current
- **📷 DSLR/Mirrorless**: 2-8e⁻ read noise, 0.05-1e⁻/px/s dark current  
- **🔭 Cooled Astro**: 1-3e⁻ read noise, <0.01e⁻/px/s dark current
- **🎬 Cinema Camera**: 1-5e⁻ read noise, optimized for video

**Real-World Examples:**
- **Daylight photo**: Shot limited, SNR >100, excellent quality
- **Indoor portrait**: Mixed regime, SNR 20-50, good quality  
- **Night photography**: Read/dark limited, SNR 5-20, challenging
- **Deep space**: Dark limited, requires long exposures + cooling

---

*This enhanced calculator now shows the complete story from photons to final image quality, including the fundamental noise limits that determine what you can photograph!*